<a href="https://colab.research.google.com/github/is0280fp/google_colab/blob/use_image_as_input/GaitRec_get_csv_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import csv
import tensorflow as tf
import functools
import numpy as np

# from IPython.core.debugger import Pdb; Pdb().set_trace()

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_V_PRO_left.csv') as f:
    reader = csv.reader(f)
    V_PRO_left = [row for row in reader]

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_V_PRO_right.csv') as f:
    reader = csv.reader(f)
    V_PRO_right = [row for row in reader]

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_AP_PRO_left.csv') as f:
    reader = csv.reader(f)
    AP_PRO_left = [row for row in reader]

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_AP_PRO_right.csv') as f:
    reader = csv.reader(f)
    AP_PRO_right = [row for row in reader]

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_ML_PRO_left.csv') as f:
    reader = csv.reader(f)
    ML_PRO_left = [row for row in reader]

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_F_ML_PRO_right.csv') as f:
    reader = csv.reader(f)
    ML_PRO_right = [row for row in reader]

with open('/content/drive/MyDrive/GaitRect/GaitRect/GRF_metadata.csv') as f:
    reader = csv.reader(f)
    metadata = [row for row in reader]

In [ ]:
meta_items = np.array(metadata, dtype=str)[0,::]

V_PRO_right_items = np.array(V_PRO_right, dtype=str)[0,::]
V_PRO_right = np.array(V_PRO_right)[1:,::]
V_PRO_right = np.array(V_PRO_right, dtype=float)

V_PRO_left_items = np.array(V_PRO_left, dtype=str)[0,::]
V_PRO_left = np.array(V_PRO_left)[1:,::]
V_PRO_left = np.array(V_PRO_left, dtype=float)

AP_PRO_right_items = np.array(AP_PRO_right, dtype=str)[0,::]
AP_PRO_right = np.array(AP_PRO_right)[1:,::]
AP_PRO_right = np.array(AP_PRO_right, dtype=float)

AP_PRO_left_items = np.array(AP_PRO_left, dtype=str)[0,::]
AP_PRO_left = np.array(AP_PRO_left)[1:,::]
AP_PRO_left = np.array(AP_PRO_left, dtype=float)

ML_PRO_right_items = np.array(ML_PRO_right, dtype=str)[0,::]
ML_PRO_right = np.array(ML_PRO_right)[1:,::]
ML_PRO_right = np.array(ML_PRO_right, dtype=float)

ML_PRO_left_items = np.array(ML_PRO_left, dtype=str)[0,::]
ML_PRO_left = np.array(ML_PRO_left)[1:,::]
ML_PRO_left = np.array(ML_PRO_left, dtype=float)

In [ ]:
# meta = np.array(meta, dytpe=float) str型や日付など変換できない列があるので、逐次floatに変換忘れない
# metadataの前処理, 3(class_label_detail)と16(year.date)列目削除して
metadata_array = np.array(metadata)
meta = np.array(metadata)[1:,::]
meta = np.delete(meta, [3,16], axis=1)
# 2列目を5クラスに対応した数字に変換(HC=0, H=1, K=2, A=3, C=4)
meta = np.where(meta=='HC', 0, meta)
meta = np.where(meta=='H', 1, meta)
meta = np.where(meta=='K', 2, meta)
meta = np.where(meta=='A', 3, meta)
meta = np.where(meta=='C', 4, meta)
# meta.shape (8971, 18)
# NaNがあるとint変換できないので、NaNをチェックする?
meta_list = []
for i in range(len(meta)):
  meta_list.append(meta[i,::].astype(np.float32))
meta = np.array(meta_list) # meta.shape (8971, 18)

In [ ]:
# GRF_and_metaはGRFにmetadataを3と16項目目を削除してものを足しただけ
# 75554~75566まで残す
valid_AP_PRO_left = AP_PRO_left[75554, ::].reshape(1, 104)
valid_AP_PRO_right = AP_PRO_right[75554, ::].reshape(1, 104)

valid_V_PRO_left = V_PRO_left[75554, ::].reshape(1, 104)
valid_V_PRO_right = V_PRO_right[75554, ::].reshape(1, 104)

valid_ML_PRO_left = ML_PRO_left[75554, ::].reshape(1, 104)
valid_ML_PRO_right = ML_PRO_right[75554, ::].reshape(1, 104)

In [ ]:
meta_items

array(['SUBJECT_ID', 'SESSION_ID', 'CLASS_LABEL', 'CLASS_LABEL_DETAILED',
       'SEX', 'AGE', 'HEIGHT', 'BODY_WEIGHT', 'BODY_MASS', 'SHOE_SIZE',
       'AFFECTED_SIDE', 'SHOD_CONDITION', 'ORTHOPEDIC_INSOLE', 'SPEED',
       'READMISSION', 'SESSION_TYPE', 'SESSION_DATE', 'TRAIN',
       'TRAIN_BALANCED', 'TEST'], dtype='<U20')

In [ ]:
# GRF_and_meta[75472::,0] == GRF[75472::,0] これより、GRF[75473,::]から最後までがいらないデータ
# metadataにないデータをSESSION_IDにもとづいて削除する(888812536~888812635と999910365~999910375の連番、ただし888812544は除く)
# strategy: 888812544だけ取り出して保存しておくが、それ以外remove
temp_AP_PRO_left = AP_PRO_left
temp_AP_PRO_left = temp_AP_PRO_left[:75473, ::] # 保存しておいたのを足す
temp_AP_PRO_left = np.append(temp_AP_PRO_left, valid_AP_PRO_left, axis=0)

temp_AP_PRO_right = AP_PRO_right
temp_AP_PRO_right = temp_AP_PRO_right[:75473, ::] # 保存しておいたのを足す
temp_AP_PRO_right = np.append(temp_AP_PRO_right, valid_AP_PRO_right, axis=0)

temp_ML_PRO_left = ML_PRO_left
temp_ML_PRO_left = temp_ML_PRO_left[:75473, ::] # 保存しておいたのを足す
temp_ML_PRO_left = np.append(temp_ML_PRO_left, valid_ML_PRO_left, axis=0)

temp_ML_PRO_right = ML_PRO_right
temp_ML_PRO_right = temp_ML_PRO_right[:75473, ::] # 保存しておいたのを足す
temp_ML_PRO_right = np.append(temp_ML_PRO_right, valid_ML_PRO_right, axis=0)

temp_V_PRO_left = V_PRO_left
temp_V_PRO_left = temp_V_PRO_left[:75473, ::] # 保存しておいたのを足す
temp_V_PRO_left = np.append(temp_V_PRO_left, valid_V_PRO_left, axis=0)

temp_V_PRO_right = V_PRO_right
temp_V_PRO_right = temp_V_PRO_right[:75473, ::] # 保存しておいたのを足す
temp_V_PRO_right = np.append(temp_V_PRO_right, valid_V_PRO_right, axis=0)

In [ ]:
# 本番
V_PRO_left = V_PRO_left[:75473, ::]
V_PRO_left = np.append(V_PRO_left, valid_V_PRO_left, axis=0)

V_PRO_right = V_PRO_right[:75473, ::]
V_PRO_right = np.append(V_PRO_right, valid_V_PRO_right, axis=0)

AP_PRO_left = AP_PRO_left[:75473, ::]
AP_PRO_left = np.append(AP_PRO_left, valid_AP_PRO_left, axis=0)

AP_PRO_right = AP_PRO_right[:75473, ::]
AP_PRO_right = np.append(AP_PRO_right, valid_AP_PRO_right, axis=0)

ML_PRO_left = ML_PRO_left[:75473, ::]
ML_PRO_left = np.append(ML_PRO_left, valid_ML_PRO_left, axis=0)

ML_PRO_right = ML_PRO_right[:75473, ::]
ML_PRO_right = np.append(ML_PRO_right, valid_ML_PRO_right, axis=0)

In [ ]:
i = 0

meta_SESSION_ID = meta[::,1]
meta_SUBJECT_ID = meta[::,0]

V_PRO_right_SESSION_ID = V_PRO_right[::,1]
V_PRO_right_and_meta = np.empty((len(V_PRO_right), 120))

V_PRO_left_SESSION_ID = V_PRO_left[::,1]
V_PRO_left_and_meta = np.empty((len(V_PRO_left), 120))

AP_PRO_right_SESSION_ID = AP_PRO_right[::,1]
AP_PRO_right_and_meta = np.empty((len(AP_PRO_right), 120))

AP_PRO_left_SESSION_ID = AP_PRO_left[::,1]
AP_PRO_left_and_meta = np.empty((len(AP_PRO_left), 120))

ML_PRO_right_SESSION_ID = ML_PRO_right[::,1]
ML_PRO_right_and_meta = np.empty((len(ML_PRO_right), 120))

ML_PRO_left_SESSION_ID = ML_PRO_left[::,1]
ML_PRO_left_and_meta = np.empty((len(ML_PRO_left), 120))

In [ ]:
for i in range(len(meta_SUBJECT_ID)):
  # from IPython.core.debugger import Pdb; Pdb().set_trace()
  index = np.array(np.where(AP_PRO_left_SESSION_ID == meta_SESSION_ID[i])[0]) # 問題
  index = np.array(index)
  for k in index:
    V_PRO_left_and_meta[k, ::] = np.append(V_PRO_left[k, ::], meta[i, 2::])
    V_PRO_right_and_meta[k, ::] = np.append(V_PRO_right[k, ::], meta[i, 2::])
    AP_PRO_left_and_meta[k, ::] = np.append(AP_PRO_left[k, ::], meta[i, 2::])
    AP_PRO_right_and_meta[k, ::] = np.append(AP_PRO_right[k, ::], meta[i, 2::])
    ML_PRO_left_and_meta[k, ::] = np.append(ML_PRO_left[k, ::], meta[i, 2::])
    ML_PRO_right_and_meta[k, ::] = np.append(ML_PRO_right[k, ::], meta[i, 2::])

In [ ]:
np.allclose(ML_PRO_left_and_meta[::,0:104], ML_PRO_left) # データセットに加工する前のGRFとmetadataのセット完成!
np.allclose(ML_PRO_right_and_meta[::,0:104], ML_PRO_right)

True

In [ ]:
np.allclose(AP_PRO_left_and_meta[::,0:104], AP_PRO_left) # データセットに加工する前のGRFとmetadataのセット完成!
np.allclose(AP_PRO_right_and_meta[::,0:104], AP_PRO_right)

True

In [ ]:
np.allclose(V_PRO_left_and_meta[::,0:104], V_PRO_left) # データセットに加工する前のGRFとmetadataのセット完成!
np.allclose(V_PRO_right_and_meta[::,0:104], V_PRO_right)

True

In [100]:
# GRFをV,AP,MLの順に横に並べる
# AFFECTED_SIDEによって横に並ぶ順番を決める
# trainとtestデータにわける
SUBJECT_num = len(V_PRO_right_and_meta)

AP_PRO_right_values = AP_PRO_right[::,3:]
AP_PRO_left_values = AP_PRO_left[::,3:]
ML_PRO_right_values = ML_PRO_right[::,3:]
ML_PRO_left_values = ML_PRO_left[::,3:]
V_PRO_right_values = V_PRO_right[::,3:]
V_PRO_left_values = V_PRO_left[::,3:]

# データのみ
right_GRF = np.concatenate([V_PRO_right_values, AP_PRO_right_values, ML_PRO_right_values], axis=0)
left_GRF = np.concatenate([V_PRO_left_values, AP_PRO_left_values, ML_PRO_left_values], axis=0)


In [101]:
right_GRF.shape

(226422, 101)

In [102]:
# AFFECTED_SIDEによって横に並ぶ順番を決める
import random

GRF = []
for i in range(SUBJECT_num):
  if V_PRO_right_and_meta[i, 2] > 0:    # patient
    if V_PRO_right_and_meta[i, 9] == 0: # affected_leg is left
      GRF.append(np.concatenate([left_GRF[i], right_GRF[i]]))
    elif V_PRO_right_and_meta[i, 9] == 1:  # affected_leg is right
      GRF.append(np.concatenate([right_GRF[i], left_GRF[i]]))
    else:                                  # affected_leg is both
      lst = []
      lst.append(right_GRF[0])
      lst.append(left_GRF[0])
      random.shuffle(lst)
      GRF.append(np.concatenate([lst[0],lst[1]])) 
  else:                                    # healthy control
    lst = []
    lst.append(right_GRF[0])
    lst.append(left_GRF[0])
    random.shuffle(lst)
    GRF.append(np.concatenate([lst[0],lst[1]])) 

In [103]:
np.array(GRF).shape

(75474, 202)

In [104]:
# trainとtestデータにわける
# この時点では、まだGRFとV_PRO_right_and_metaの順番は同じはず
train_data = []
test_data = []
for i in range(SUBJECT_num):
  if V_PRO_right_and_meta[i, -1] == 1: # testデータのラベル
    test_data.append(GRF[i])
  else:
    train_data.append(GRF[i])

In [105]:
import pickle

with open("train.pickle", mode="wb") as f:
   pickle.dump(train_data, f)

with open("test.pickle", mode="wb") as f:
   pickle.dump(test_data, f)

with open("train.pickle", mode="rb") as f:
   X = pickle.load(f)

with open("test.pickle", mode="rb") as f:
   y = pickle.load(f)